<a href="https://colab.research.google.com/github/seunggeon/KW_AI_Hackathon/blob/main/%EA%B4%91%EC%9A%B4%ED%95%B4%EC%BB%A4%ED%86%A4_%EC%B9%BC%EB%A7%8C%ED%95%84%ED%84%B0(%EC%9B%90%EB%B3%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Date Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/월간 11 운동')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn.cluster import KMeans
import math
from tqdm import tqdm

train = pd.read_csv('train_features.csv')
train_acc, train_gy  = train.iloc[:, 2:5], train.iloc[:, 5:] #iloc 알아두기. 데이터에 특정 행과 열을 선택.

train_label = pd.read_csv('train_labels.csv')
train_y = train_label.label

test = pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

train_gy

,gy_x,gy_y,gy_z
0,-0.591608,-30.549010,-31.676112
1,0.303100,-39.139103,-24.927216
2,-3.617278,-44.122565,-25.019629
3,2.712986,-53.597843,-27.454013
4,4.286707,-57.906561,-27.961234
...,...,...,...
1874995,-29.367857,-104.013664,-76.290437
1874996,-30.149089,-101.796809,-76.625087
1874997,-27.873095,-98.776072,-79.365125
1874998,-23.636550,-99.139495,-80.259478


In [ ]:
# GPU Check 

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Data Agument

In [ ]:
import scipy
!pip install transforms3d
from transforms3d.axangles import axangle2mat

def rolling(data):
    for j in np.random.choice(data.shape[0], int(data.shape[0]*2/3)):
        data[j] = np.roll(data[j], np.random.choice(data.shape[1]), axis= 0)
    return data

def rotation(data):
    axis = np.random.uniform(low=-1, high=1, size=data.shape[1])
    angle = np.random.uniform(low=-np.pi, high=np.pi)
    return np.matmul(data , axangle2mat(axis,angle))

def permutation(data, nPerm=4, mSL=10):
    data_new = np.zeros(data.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(mSL, data.shape[0]-mSL, nPerm-1))
        segs[-1] = data.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > mSL:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        data_temp = data[segs[idx[ii]]:segs[idx[ii]+1],:]
        data_new[pp:pp+len(data_temp),:] = data_temp
        pp += len(data_temp)
    return(data_new)

"""
sigma = 0.05
def Jitter(data, sigma=0.05):
    myNoise = np.random.normal(loc=0, scale=sigma, size=data.shape)
    return data+myNoise
"""

def combine_aug(data, k, aug_P = 0):
    data_ = data.copy()
    if aug_P == 0:
        if (k+1) % 2 == 0:
            for i in np.random.choice(int(data.shape[0]/600), int(data.shape[0]/600*2/3)):
                data_[600*i:600*(i+1)] = rotation(np.array(data_[600*i:600*(i+1)]))
        if (k+1) % 2 == 1:
            for i in np.random.choice(int(data.shape[0]/600), int(data.shape[0]/600*2/3)):
                data_[600*i:600*(i+1)] = permutation(np.array(data_[600*i:600*(i+1)]))
                
    if aug_P != 0:
        pass
    return data_


4/1AX4XfWi6DOpBeyUiaiZuedH9Gptp9IJYzT0SWnCK6pWaxRYcyIk7qIZ-EYMCollecting transforms3d
     |████████████████████████████████| 62 kB 597 kB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-py3-none-any.whl size=59373 sha256=c38eeee6721a55a8c7b50dcd83312f1899c4d8b62b00b19efb1d87aba0923c97
  Stored in directory: /root/.cache/pip/wheels/b5/b7/93/8985551f83720ce37548a5b543c75380bb707955a9c2c5d28c
Successfully built transforms3d


## Data transform
Eolier Angle and Energy 

In [ ]:
pi = math.pi
dt=0.02 
from math import atan, sqrt

def roll_pitch(data):
    roll = (data.iloc[:,1]/(data.iloc[:,0]**2 + data.iloc[:,2]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi
    pitch = (data.iloc[:,0]/(data.iloc[:,1]**2 + data.iloc[:,2]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi
    return pd.concat([roll,pitch], axis=1)

# 원본데이터를 에너지값으로 표현
def get_energy(data):
    energy_ = (data.iloc[:,0]**2+data.iloc[:,1]**2+data.iloc[:,2]**2)**(1/3)
    return energy_ 



## Kalman Filter

In [ ]:
 '''
 Filename: 11_EulerKalman.py
 Created on: April,3, 2021
 Author: dhpark
 From: 칼만필터는 어렵지 않아 / https://github.com/DonghunP/Book_KalmanFilter
'''

def GetAccel(i):
    # (41500, 1)
    return ax, ay, az

def EulerAccel(ax, ay, az):
    theta = atan(ax/sqrt(ay**2 + az**2))
    phi = atan(ay/az)
    return phi, theta

def EulerToQuaternion(phi, theta, psi):
    sinPhi = sin(phi/2)
    cosPhi = cos(phi/2)
    sinTheta = sin(theta/2)
    cosTheta = cos(theta/2)
    sinPsi = sin(psi/2)
    cosPsi = cos(psi/2)
    z = np.array([cosPhi*cosTheta*cosPsi + sinPhi*sinTheta*sinPsi,
                  sinPhi*cosTheta*cosPsi - cosPhi*sinTheta*sinPsi,
                  cosPhi*sinTheta*cosPsi + sinPhi*cosTheta*sinPsi,
                  cosPhi*cosTheta*sinPsi - sinPhi*sinTheta*cosPsi])
    return z

def EulerKalman(A, z):
    global firstRun
    global Q, H, R
    global x, P
    if firstRun:
        H = np.eye(4)
        Q = 0.0001 * np.eye(4)
        R = 10 * np.eye(4)
        x = np.array([1, 0, 0, 0]).transpose()
        P = np.eye(4)
        firstRun = False
    else:
        Xp = A @ x # Xp : State Variable Prediction
        Pp = A @ P @ A.T + Q # Error Covariance Prediction

        K = (Pp @ H.T) @ inv(H@Pp@H.T + R) # K : Kalman Gain

        x = Xp + K@(z - H@Xp) # Update State Variable Estimation
        P = Pp - K@H@Pp # Update Error Covariance Estimation

    phi   = atan2(2 * (x[2] * x[3] + x[0] * x[1]), 1 - 2*(x[1]**2 + x[2]**2))
    try:
      theta = -asin(2 *  (x[1] * x[3] - x[0] * x[2]))
    except ValueError:
      theta = 0
    psi   = atan2(2 *  (x[1] * x[2] + x[0] * x[3]), 1-2*(x[2]**2 + x[3]**2))
    return phi, theta, psi

def kalman_excute(data_acc,data_gy):

  import numpy as np
  from numpy.linalg import inv
  import matplotlib.pyplot as plt
  from math import cos, sin, asin, atan2, pi, atan, sqrt
  from scipy import io

  H, Q, R = None, None, None  
  x, P = None, None
  firstRun = True

  Nsamples = 600
  EulerSaved = np.zeros([Nsamples,3])
  dt = 0.01

  for k in range(Nsamples):
    p = data_gy.iloc[k,0]  
    q = data_gy.iloc[k,1]  
    r = data_gy.iloc[k,2] 

    A = np.eye(4) + dt * (1/2) * np.array([[0,-p,-q,-r],[p,0,r,-q],[q,-r,0,p],[r,q,-p,0]])
    ax = data_acc.iloc[k,0]   # (41500, 1)
    ay = data_acc.iloc[k,1]  # (41500, 1)
    az = data_acc.iloc[k,2] 
    phi, theta = EulerAccel(ax, ay, az)
    z = EulerToQuaternion(phi, theta, 0) #State variable as Quaternion form

    phi, theta, psi = EulerKalman(A, z)
    EulerSaved[k] = [phi, theta, psi]

  t = np.arange(0, Nsamples * dt ,dt)
  PhiSaved = EulerSaved[:,0] * 180/pi
  ThetaSaved = EulerSaved[:,1] * 180/pi
  PsiSaved = EulerSaved[:,2] * 180/pi

  PhiSaved = pd.DataFrame(PhiSaved)
  ThetaSaved = pd.DataFrame(ThetaSaved)
  
  return pd.concat([PhiSaved,ThetaSaved], axis=1)

"\nplt.figure()\nplt.plot(t, PhiSaved)\nplt.xlabel('Time [Sec]')\nplt.ylabel('Roll angle [deg]')\nplt.savefig('11_EulerKalman_roll.png')\n\nplt.figure()\nplt.plot(t, ThetaSaved)\nplt.xlabel('Time [Sec]')\nplt.ylabel('Pitch angle [deg]')\nplt.savefig('11_EulerKalman_pitch.png')\nplt.show()\n"

## Train_dataset & Test Dataset

In [ ]:
def train_dataset(acc_data, gy_data, i, aug_P = 0):

    aug_acc = combine_aug(acc_data, i, aug_P)
    aug_gy = combine_aug(gy_data, i, aug_P)
    
    energy_acc = get_energy(aug_acc)
    energy_gy  = get_energy(aug_gy)

    kalam_data = kalman_excute(acc_data, gy_data)

    return pd.concat([aug_acc, kalam_data, energy_acc, energy_gy ,
                     aug_gy], axis= 1)

def test_dataset(acc_data, gy_data):
    
    energy_acc = get_energy(acc_data)
    energy_gy  = get_energy(gy_data)

    kalam_data = kalman_excute(acc_data, gy_data)

    return pd.concat([acc_data, kalam_data, energy_acc, energy_gy,
                      gy_data], axis= 1)

In [ ]:

import sklearn
from sklearn.preprocessing import StandardScaler

data_for_scaler = train_dataset(train_acc, train_gy,1) # train data만 사용
scaler = StandardScaler().fit(np.array(data_for_scaler))

data_for_scaler = np.array(data_for_scaler).reshape(-1, 600, data_for_scaler.shape[1])
########################################################################################
test_x = test_dataset(test.iloc[:, 2:5], test.iloc[:, 5:])

test_X = scaler.transform(np.array(test_x)).reshape(-1, 600, test_x.shape[1])


# Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


In [ ]:
'''
import tensorflow as tf
import tensorflow.keras as keras
!pip install tensorflow-addons
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras
'''

'\nimport tensorflow as tf\nimport tensorflow.keras as keras\n!pip install tensorflow-addons\nimport tensorflow_addons as tfa\nfrom keras.models import Sequential\nfrom keras.layers import Dense, LSTM,Bidirectional,Dropout\nfrom keras.layers import Dense\nfrom keras.layers import Dropout\nfrom keras.layers import LSTM\nfrom keras.layers.convolutional import Conv1D\nfrom keras.layers.convolutional import MaxPooling1D\nfrom keras.utils import to_categorical\nfrom keras import backend as K \nfrom keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau\nfrom sklearn.model_selection import KFold,StratifiedKFold\nfrom numpy.random import seed\nimport keras\n'

## GRU Model

In [ ]:
def first_model():
    inputs = L.Input(shape = (data_for_scaler.shape[1], data_for_scaler.shape[2]))
    gru1 = L.GRU(256, return_sequences = True, dropout = 0.2)(inputs)
    mp = L.MaxPool1D()(gru1)
    ap = L.AveragePooling1D()(gru1)
    concat1 = L.Concatenate()([mp, ap])
    gru2 = L.GRU(256, return_sequences = True, dropout = 0.2)(concat1)
    GAP = L.GlobalAveragePooling1D()(gru2)
    dense = L.Dense(61, activation = "softmax")(GAP)
    return keras.models.Model(inputs, dense)

## CNN model

In [ ]:
def cnn_model(input_shape, classes):
    seed(2021)
    tf.random.set_seed(2021)
    
    input_layer = keras.layers.Input(input_shape)
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

# Train Model

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
import sklearn

def train_model(model_ = None, epochs = 40, first_rlr = 15, second_rlr = 30, r_seed = 10, aug_P = 0, seed_ = 1):
    # first_rlr : 첫번째로 learning_rate이 감소
    # second_rlr : 두번째로 learning_rate이 감소
    # r_seed : StratifiedKFold seed
    # seed_ : numpy/random seed
    
    result_model = []
    cnt = 0
    train_reshape_acc = np.array(train_acc).reshape(-1, 600, 3)
    train_reshape_gy = np.array(train_gy).reshape(-1, 600, 3)
    
    random.seed(seed_)
    tf.random.set_seed(21)

    spliter = StratifiedKFold(n_splits=10, shuffle = True, random_state = r_seed)
    for train_split_id, valid_split_id in spliter.split(train_reshape_acc, train_y):
        
        train_Y, valid_Y = np.array(pd.get_dummies(train_label.label))[train_split_id], np.array(pd.get_dummies(train_label.label))[valid_split_id]

        valid_ACC, valid_GY = train_reshape_acc[valid_split_id].reshape(-1, 3), train_reshape_gy[valid_split_id].reshape(-1, 3)
        valid_x = test_dataset(pd.DataFrame(valid_ACC), pd.DataFrame(valid_GY))
        valid_X = scaler.transform(np.array(valid_x)).reshape(-1, 600, valid_x.shape[1])

        model = model_()
        model.compile(optimizer=keras.optimizers.RMSprop(0.003),
                      loss='categorical_crossentropy', metrics=['accuracy'])
        val_score = 0
        seed_ += 1

        for i in range(epochs):
            
            np.random.seed(seed_*47 + i)
            
            train_ACC, train_GY = train_reshape_acc[train_split_id].reshape(-1, 3), train_reshape_gy[train_split_id].reshape(-1, 3)
            train_x = train_dataset(pd.DataFrame(train_ACC), pd.DataFrame(train_GY), i, aug_P)
            train_X = scaler.transform(np.array(train_x)).reshape(-1, 600, valid_x.shape[1])

            train_X_ = train_X.copy()

            train_X_ = rolling(train_X_)

            hist = model.fit(train_X_, train_Y, epochs = 1, validation_data = (valid_X, valid_Y), verbose = 0)

            train_accuracy = hist.history["accuracy"]
            new_val_score = accuracy_score(np.argmax(valid_Y, axis = 1), np.argmax(model.predict(valid_X), axis = 1))
            val_loss = hist.history["val_loss"]

            if i == first_rlr:
                model.compile(optimizer=keras.optimizers.RMSprop(0.003*0.2),
                              loss='categorical_crossentropy', metrics=['accuracy'])

            if i == second_rlr:
                model.compile(optimizer = keras.optimizers.RMSprop(0.003*0.2*0.4),
                             loss='categorical_crossentropy', metrics=['accuracy'])

            print("epoch {} - train_accuracy : {} - validation_loss : {} - validation_accuracy : {}".format(i,
                                                                                                            train_accuracy,
                                                                                                            val_loss,
                                                                                                            new_val_score,
                                                                                                            ))

            if i == 0:
                val_loss_score = val_loss[0]
        
            if val_loss_score >= val_loss[0]:
                val_loss_score = val_loss[0]
                best_model = model
                print("####best_val####")
                    
            if new_val_score >= val_score:
                val_score = new_val_score
                best_model = model
                print("####best_acc####")
        print("####################################################### cycle {} is done".format(cnt))
        result_model.append(best_model)
        cnt+=1

        
    return result_model


def predict_(model):
    result = []
    for mod in model:
        result.append(mod.predict(test_X))
    predict = np.array(result).mean(axis = 0)

    return predict

def save_model(models, name = '1'):
    cnt = 1
    for model in models:
        model.save(path + "submission/last/weight/" + name + '-{}.h5'.format(cnt))
        cnt +=1


# Test Model

In [ ]:
first_result = train_model(first_model, r_seed = 47, seed_ = 1)

epoch 0 - train_accuracy : [0.003911806736141443] - validation_loss : [nan] - validation_accuracy : 0.003194888178913738
####best_acc####
epoch 1 - train_accuracy : [0.003911806736141443] - validation_loss : [nan] - validation_accuracy : 0.003194888178913738
####best_acc####
epoch 2 - train_accuracy : [0.003911806736141443] - validation_loss : [nan] - validation_accuracy : 0.003194888178913738
####best_acc####
epoch 3 - train_accuracy : [0.003911806736141443] - validation_loss : [nan] - validation_accuracy : 0.003194888178913738
####best_acc####
epoch 4 - train_accuracy : [0.003911806736141443] - validation_loss : [nan] - validation_accuracy : 0.003194888178913738
####best_acc####
epoch 5 - train_accuracy : [0.003911806736141443] - validation_loss : [nan] - validation_accuracy : 0.003194888178913738
####best_acc####
epoch 6 - train_accuracy : [0.003911806736141443] - validation_loss : [nan] - validation_accuracy : 0.003194888178913738
####best_acc####
epoch 7 - train_accuracy : [0.0039

# Save Result

In [ ]:
submission.to_csv('baseline_submission.csv', index=False)